In [11]:
import os
import cv2
import pyewts
import numpy as np
from tqdm import tqdm
from evaluate import load
from Utils import read_label
from Modules import OCRInference

In [2]:
ocr_model_config = "Models/OCRModels/LhasaKanjur/model_config_v2.json"
ocr_inference = OCRInference(config_file=ocr_model_config)
converter = pyewts.pyewts()
cer_scorer = load("cer")

In [3]:
dataset_path = "D:/Datasets/Tibetan/LhasaKanjur/LhasaKanjurProdigy_v1_reviewed"
test_set = "Data/LhasaKanjur/test_imgs.txt"

def get_entries(file_path: str, test_data_path: str):
    with open(test_set, "r", encoding="utf-8") as f:
        entries = f.readlines()
        entries = [x.replace("\n", "") for x in entries]
        entries = [os.path.basename(x) for x in entries]
        entries = [x.split(".")[0] for x in entries]

        images = [f"{test_data_path}/lines/{x}.jpg" for x in entries]
        labels = [f"{test_data_path}/transcriptions/{x}.txt" for x in entries]

    return images, labels


images, labels = get_entries(test_set, dataset_path)


In [8]:
test_idx = 123
test_image = images[test_idx]
test_label = labels[test_idx]

test_lbl = read_label(test_label)
test_img = cv2.imread(test_image)

wylie_label = converter.toWylie(test_lbl)
predicted_text, _ = ocr_inference.run([test_img], replace_tseg=True)


print(f"Label: {wylie_label}")
print(f"Prediction: {predicted_text[0]}")

Label: gcig sbyin pa yang ma thob/_phyir gtong ba yang ma thob par phyir byung ba las na/_'dir bcom ldan 'das kyi nyan thos/_byis pa/_blun pa/_mi gsal ba/_mi mkhas pa 'ga' zhig gis nyon mongs 
Prediction: gcig sbyin pa yang ma thob/ phyir gtong ba yang ma thob par phyir byung ba las na/ 'dir bcom ldan 'das kyi nyan thos/ byis pa/ blun pa/ mi gsal ba/ mi mkhas pa 'ga' zhig gis nyon mongs 


In [10]:
cer_scores = []

for image, label in tqdm(zip(images, labels), total=len(images)):

    image = cv2.imread(image)
    lbl = read_label(label)
    wylie_label = converter.toWylie(lbl)
    predicted_text, raw_prediction = ocr_inference.run([image], replace_tseg=True)

    cer_score = cer_scorer.compute(predictions=predicted_text, references=[wylie_label])
    cer_scores.append(cer_score)

100%|██████████| 16064/16064 [20:34<00:00, 13.01it/s] 


In [5]:
mean_cer = np.mean(cer_scores)
print(f"Mean CER: {mean_cer}")

16064
[0.04, 0.011363636363636364, 0.029940119760479042, 0.005747126436781609, 0.030303030303030304, 0.03278688524590164, 0.03896103896103896, 0.08152173913043478, 0.010582010582010581, 0.017241379310344827, 0.025974025974025976, 0.015463917525773196, 0.03902439024390244, 0.027777777777777776, 0.041916167664670656, 0.022099447513812154, 0.02631578947368421, 0.032432432432432434, 0.01875, 0.018518518518518517, 0.023809523809523808, 0.016216216216216217, 0.015957446808510637, 0.01935483870967742, 0.016483516483516484, 0.021164021164021163, 0.010471204188481676, 0.03684210526315789, 0.0189873417721519, 0.03867403314917127, 0.029940119760479042, 0.03225806451612903, 0.017543859649122806, 0.016574585635359115, 0.03888888888888889, 0.02247191011235955, 0.00558659217877095, 0.046242774566473986, 0.030303030303030304, 0.031055900621118012, 0.029411764705882353, 0.0425531914893617, 0.022857142857142857, 0.015503875968992248, 0.03977272727272727, 0.05263157894736842, 0.055944055944055944, 0.0103